# TezaAI programming assignment

To run this notebook, following steps are needed:

Requirement:
1. Python 3.6.5

Open Terminal

1. Navigate to the project folder (cd {downloaded_file}/teza_ai) 
2. Activate the virtual environment (. ./setup-env.sh)
3. Install the dependepencies (pip install -r requirements.txt)
4. Start "Jupyter Notebook" (jupyter notebook)
5. Click on the notebook "TezaAI Programming Assignment"

In [1]:
# Libraries import

import cerberus
import json
import os
import re
import numpy as np
import string
from collections import Counter 
from pprint import pprint

## Input Files Validation schema

This peace of code below validates the input files format.
i.e. if they are valid files.

In [2]:
# building validation dict based on the structure below

# {
#  "_id": "https://www.ekmhinnovators.com/ekmh-innovators-blog-beta/interview-ourcrowd-ceo-jon-medved-on-impact-investing-crowdfunding",
#  "title": "Interview: OurCrowd CEO Jon Medved on Crowdfunding, Beyond ...",
#  "body": "EKMH Innovators Interview Series An interview ...",
#  "origin": "google custom search",
#  "feedId": 103,
#  "jobId": "37b3e04c-cf7d-4032-82ad-a2bd89dc90ac",
#  "person": {
# 	 "id": "16",
# 	 "name": "Jon Medved"
#  }
# }

STRING_MANDATORY = {'type': 'string', 'empty': False, 'required': True}
INT_MANDATORY = {'type': 'integer', 'empty': False, 'required': True}
VALIDATION_SCHEMA = {
    '_id': STRING_MANDATORY, 
    'title': STRING_MANDATORY, 
    'body': STRING_MANDATORY, 
    'origin': STRING_MANDATORY, 
    'feedId': INT_MANDATORY, 
    'jobId': STRING_MANDATORY, 
    'person': {
        'type': 'dict', 'required': True, 'empty': False, 
            'schema': {
                'id': STRING_MANDATORY,
                'name': STRING_MANDATORY
            }
        }
    }

In [3]:
# displaying validation dictionary

VALIDATION_SCHEMA

{'_id': {'type': 'string', 'empty': False, 'required': True},
 'title': {'type': 'string', 'empty': False, 'required': True},
 'body': {'type': 'string', 'empty': False, 'required': True},
 'origin': {'type': 'string', 'empty': False, 'required': True},
 'feedId': {'type': 'integer', 'empty': False, 'required': True},
 'jobId': {'type': 'string', 'empty': False, 'required': True},
 'person': {'type': 'dict',
  'required': True,
  'empty': False,
  'schema': {'id': {'type': 'string', 'empty': False, 'required': True},
   'name': {'type': 'string', 'empty': False, 'required': True}}}}

In [4]:
# Data global var

DATA = None

In [5]:
"""
validate input data schema
"""
def validate_schema(entities, schema, filepath):
    checker = cerberus.Validator()
    checker.allow_unknown = True
    if checker.validate(entities, schema):
        print(f'Input schema validated for {filepath} !')
    else:
        errors = checker.errors
        proceed = False
        error = errors
        raise ValueError(f'Format mismatch for the input {errors}')

In [6]:
"""
Validate and read the JSON data
file_path: path of the json file
"""
def read_and_validate_data(filepath):
    with open(filepath) as json_file:
        data = json.load(json_file)
        validate_schema(entities=data, schema=VALIDATION_SCHEMA,\
                        filepath=filepath)
    return data

Before executing the block of code below, more files (interview JSONs) can be added to the data folder

The data folder with the source code comprises of the already provided 6 JSON input interview files



In [7]:
# Path of the data files
# Add more files to the project folder or change this path to read from any 
# other directory path
DATA_FOLDER = os.path.join(os.path.curdir, 'data') # by default it takes the JSONs from project folder

# only considering JSON files at the moment
interview_files_list = [os.path.join(DATA_FOLDER, f) for f in os.listdir(DATA_FOLDER) \
                        if '.json' in os.path.splitext(f) ]


print(interview_files_list)
if len(interview_files_list) < 1:
    proceed = False
    error = "No data file available"

['./data/0.json', './data/1.json', './data/2.json', './data/3.json', './data/4.json', './data/5.json']


In [8]:
# Reading in the data input files while also 
# validating them
DATA = [read_and_validate_data(file) for file in interview_files_list]

Input schema validated for ./data/0.json !
Input schema validated for ./data/1.json !
Input schema validated for ./data/2.json !
Input schema validated for ./data/3.json !
Input schema validated for ./data/4.json !
Input schema validated for ./data/5.json !


## Common programming methods

Let us define some common methods before progressing with interview quotes extraction

In [9]:
"""
Remove special character except the exceptions list
"""
def remove_special_character(text_to_process, exceptions=[], strip=False):
    new_string = str(text_to_process)
    if strip:
        new_string = new_string.strip()
    special_chars = string.printable[62:]
    special_chars = [x for x in special_chars if x not in exceptions]
    for char in text_to_process:
        if char in special_chars:
            new_string = new_string.replace(char, '')
    return new_string

In [10]:
"""
Remove special character without spaces
"""
def remove_special_character_without_spaces(text_to_process, exceptions=[]):
    return remove_special_character(text_to_process, [' '], strip=True)

In [11]:
"""
Remove spaces
"""
def remove_spaces(text_to_process):
    return text_to_process.replace(" ", "")

In [12]:
"""
Reverse the array
"""
def reverse_the_array(arr):
    return arr[::-1]

## Custom variables and methods for finding Interviewee and Interviewers

In [13]:
REGULAR_EXPRESSION = '(?<=\{})(.*?)(?=\:)' # to match the names with article


# article generally ends with a fullstop, question mark or semicolon..
# add more to list if required
INTERVIEW_ENDING = ['.', '?', ";"]  


# Name threshold
THRESHOLD = 20

In [14]:
"""
Predicts the interviewee names based on the information
provided in the json metadata while mapping it with the 
data


Assumption: assuming that there is only interviewee
"""

def predict_interviewee(name_doc_mapping={}):
    predicted_interviewee = {}
    for key, value in name_doc_mapping.items():
        item_list = max(value.items(), key=lambda x: x[1])
        labels = list()
        for k, v in value.items():
            if v == item_list[1]:
                predicted_interviewee[key] = k
    return predicted_interviewee

In [15]:
"""
Getting occurence of name in the interview text
"""

def element_occurence(name, interview_text):
    return len([pos.start() for pos in re.finditer(f'{name}:', interview_text)])

# In cell interpretation

In [19]:
# Approach 1: Interviewee name based on name provided in JSON
# And using that info evaluate what form of name is used
# e.g. for a name Tom Hilton, in the article only "Tom" could be used

# ------------------------------------------------------------------
# in-cell compilation of logic ..... Fetch names based 
# ------------------------------------------------------------------

doc = {}

for idx, entity in enumerate(DATA):
    
    name_dict = {}
    
    interview_text = str.lower(entity.get('body')) # interview content
    interviewee = entity.get('person')['name'] # person being interviewed
    
    sub_names = interviewee.split()
    sub_names.append(interviewee)
    names = [x.lower() for x in sub_names] # name list because only part of the name could be used
    
    
    # create of a dictionary to map interviewer names used in the article
    for name in names:
        name_dict[name] = element_occurence(name, interview_text)
    doc[f'doc_{idx}'] = name_dict



In [20]:
doc

{'doc_0': {'joe': 23, 'lonsdale': 0, 'joe lonsdale': 0},
 'doc_1': {'balaji': 0, 'srinivasan': 13, 'balaji srinivasan': 0},
 'doc_2': {'jon': 0, 'medved': 11, 'jon medved': 11},
 'doc_3': {'phil': 0, 'libin': 64, 'phil libin': 64},
 'doc_4': {'john': 0, 'mackey': 22, 'john mackey': 1},
 'doc_5': {'tim': 0, 'cook': 43, 'tim cook': 43}}

In [21]:
# Approach 2: Interviewee & Interviewers name based on the article contents 
# and evaluating the results with the approach 1 result to find consolidated results
# ------------------------------------------------------------------
# in-cell compilation of logic ..... Fetch article based entities
# ------------------------------------------------------------------

doc_exprsn = {}

for idx, entity in enumerate(DATA):
    
    count_dict = {}
    all_matches = []
    
    interview_text = str.lower(entity.get('body')) # interview content
    
    # Now we will try to create the same kind of map for the interviewers & interviewees based
    # on article contents
    for ex in INTERVIEW_ENDING:
        expression = REGULAR_EXPRESSION.format(ex)
        matches = re.findall(expression, interview_text)
        all_matches = all_matches + matches
    all_matches = list(map(remove_special_character_without_spaces, all_matches))
    counter = Counter(all_matches)
    for elm in set(all_matches):
        if len(elm) < THRESHOLD:
            count_dict[elm] = counter[elm]
    doc_exprsn[f'doc_{idx}'] = count_dict




In [28]:
count = 0
article_name_dict = {}
for key, value in doc_exprsn.items():
    temp_dict = {}
    name_keys = value.keys()
    print(name_keys)
    for name_key in name_keys:
        temp_dict[name] = element_occurence(name_key, str.lower(DATA[count].get('body')))
        
    
    article_name_dict[f'doc_{count}'] = temp_dict
    count = count + 1
    

dict_keys(['joe', '” then they say', 'martin'])
dict_keys(['jackson', 'balaji s srinivasan', 'srinivisan', 'srinivasan'])
dict_keys(['jon medved', 'ekmh'])
dict_keys(['laughter phil libin', 'yeah nicole torres', 'phil libin', 'nicole torres'])
dict_keys(['mackey', 'reason', ' mackey', 'john mackey'])
dict_keys(['tim cook', 'charlie rose', 'phil schiller', ' tim cook', 'graham townsend', 'angela ahrendts', 'jony ive'])


In [29]:
article_name_dict

{'doc_0': {'tim cook': 24},
 'doc_1': {'tim cook': 13},
 'doc_2': {'tim cook': 12},
 'doc_3': {'tim cook': 62},
 'doc_4': {'tim cook': 1},
 'doc_5': {'tim cook': 13}}

In [46]:
# # in cell compilation of logic .....


# doc = {}
# doc_exprsn = {}

# for idx, entity in enumerate(data):
    
#     count_dict = {}
#     name_dict = {}
#     all_matches = []
    
#     interview_text = str.lower(entity.get('body')) # interview content
    
    
#     interviewee = entity.get('person')['name'] # person being interviewed
    
#     sub_names = interviewee.split()
#     sub_names.append(interviewee)
#     names = [x.lower() for x in sub_names] # name list because only part of the name could be used
    
    
#     # create of a dictionary to map interviewer names used in the article
#     for name in names:
#         name_dict[name] = element_occurence(name, interview_text)
#     doc[f'doc_{idx}'] = name_dict
    
#     # Now we will try to create the same kind of map for the interviewers
#     for ex in INTERVIEW_ENDING:
#         expression = REGULAR_EXPRESSION.format(ex)
#         matches = re.findall(expression, interview_text)
#         all_matches = all_matches + matches
#     all_matches = list(map(remove_special_character_without_spaces, all_matches))
#     counter = Counter(all_matches)
#     for elm in set(all_matches):
#         if len(elm) < THRESHOLD:
#             count_dict[elm] = counter[elm]
#     doc_exprsn[f'doc_{idx}'] = count_dict


## Predicting Interviewers and Interviewees

In [47]:
print('Predicting Interviewers names used in the article .......\n')
pprint(predict_interviewee(doc))

Predicting Interviewers names used in the article .......

{}


In [48]:
doc_exprsn

{'doc_0': {'martin': 2, 'joe': 16, '” then they say': 1},
 'doc_1': {'jackson': 2,
  'balaji s srinivasan': 1,
  'srinivisan': 1,
  'srinivasan': 12},
 'doc_2': {'jon medved': 8, 'ekmh': 1},
 'doc_3': {'laughter phil libin': 1,
  'yeah nicole torres': 1,
  'nicole torres': 18,
  'phil libin': 53},
 'doc_4': {' mackey': 1, 'reason': 1, 'mackey': 19, 'john mackey': 1},
 'doc_5': {'tim cook': 30,
  'graham townsend': 1,
  'angela ahrendts': 1,
  'phil schiller': 1,
  'jony ive': 8,
  ' tim cook': 2,
  'charlie rose': 14}}

In [50]:
def predict_interviewers(doc_exprsn):
    potential_interviewers = {}
    for key, value in doc_exprsn.items():
        print(key, value)
        
        
#         item_list = max(value.items(), key=lambda x: x[1])
#         labels = list()
#         for k, v in value.items():
#             if v == item_list[1]:
#                 predicted_interviewee[key] = k
#     return predicted_interviewee 

In [51]:
predict_interviewers(doc_exprsn)

doc_0 {'joe': 16, 'martin': 2, 'i appreciate it martin': 1, '” then they say': 1}
doc_1 {'srinivasan': 12, 'srinivisan': 1, 'jackson': 2, 'balaji s srinivasan': 1}
doc_2 {'jon medved': 8, 'ekmh': 1}
doc_3 {'phil libin': 53, 'that too nicole torres': 1, 'yeah nicole torres': 1, 'laughter phil libin': 1, 'nicole torres': 18}
doc_4 {' mackey': 1, 'john mackey': 1, 'mackey': 19, 'reason': 1}
doc_5 {'tim cook': 30, 'angela ahrendts': 1, 'jony ive': 8, ' tim cook': 2, 'charlie rose': 14, 'graham townsend': 1, 'phil schiller': 1}


In [46]:

    

    # print('Keys with maximum Value in Dictionary : ', listOfKeys)

In [47]:
predicted_interviewee

NameError: name 'predicted_interviewee' is not defined